In [9]:
using DrWatson
@quickactivate
include(srcdir("main.jl"))
using DataFrames
using AlgebraOfGraphics

In [10]:
df = collect_results(datadir("simulations"));

┌ Info: Scanning folder /Users/zijin/projects/ion_scattering_by_SWD/data/simulations for result files.
└ @ DrWatson /Users/zijin/.julia/packages/DrWatson/8XYbJ/src/result_collection.jl:117
┌ Warning: type Main.#119#122{Float64,Float64} does not exist in workspace; reconstructing
└ @ JLD2 /Users/zijin/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type TestParticle.Field{false,JLD2.ReconstructedStatic{Symbol("#119#122{Float64,Float64}"), (:α, :β), Tuple{Float64, Float64}}} does not exist in workspace; reconstructing
└ @ JLD2 /Users/zijin/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:573
┌ Warning: type TestParticle.Field{false,Main.#E_field} does not exist in workspace; reconstructing
└ @ JLD2 /Users/zijin/.julia/packages/JLD2/9xHXH/src/data/reconstructing_datatypes.jl:457
┌ Warning: read type Tuple{Vector{Float64}, Vector{Float64}, Tuple, Float64} is not a leaf type in workspace; reconstructing
└ @ JLD2 /Users/zijin/.julia/packages/JLD2

In [ ]:
# sols, sol_field = sim(d)
# Step 4: Visualization
# plot_trajectory(d) = plot_trajectory(sim(d)...)
# f = plot_trajectory(sols, sol_field)
# plot_trajectory(d)

In [13]:
"""
Cos pitch angle
"""
function cos_pitch_angle(u, B)
    v = @view u[4:6]
    return B ⋅ v / norm(v) / norm(B)
end
    
function cos_pitch_angle(sol::ODESolution, i)
    B_field = r -> RD_B_field(r, α, β)
    B_func = sol.prob.p[3]
    u = sol.u[i]
    B = B_func(u)
    return cos_pitch_angle(u, B)
end

function d_cos_pitch_angle(sol)
    return cos_pitch_angle(sol, 1) - cos_pitch_angle(sol, length(sol))
end

function d_cos_pitch_angle(sol::EnsembleSolution)
    return map(sol) do s
        d_cos_pitch_angle(s)
    end
end

d_cos_pitch_angle (generic function with 2 methods)

In [15]:
df[!, :Δw] = d_cos_pitch_angle.(df[!, :result]);

MethodError: MethodError: objects of type JLD2.ReconstructedStatic{Symbol("Field{false,JLD2.ReconstructedStatic{Symbol(\"#119#122{Float64,Float64}\"), (:α, :β), Tuple{Float64, Float64}}}"), (:field_function,), Tuple{JLD2.ReconstructedStatic{Symbol("#119#122{Float64,Float64}"), (:α, :β), Tuple{Float64, Float64}}}} are not callable